In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyart
import glob
import matplotlib.patheffects as path_effects
from matplotlib.colors import LinearSegmentedColormap

# Radar location (La Dôle radar, Switzerland)
radar_lat = 46.425676  # Latitude of La Dôle radar
radar_lon = 6.100092   # Longitude of La Dôle radar
radar_alt = 1670       # Altitude in meters

# Coordinates of cities
geneva_lat = 46.2044
geneva_lon = 6.1432
lausanne_lat = 46.5197
lausanne_lon = 6.6323

# Load original velocity data
path_orig = '/scratch/mch/fackerma/orders/case_150619/MLD/'
file_orig = glob.glob(path_orig + 'MLD1916614300U.003')
data_orig = pyart.aux_io.read_metranet(file_orig[0], reader='python')
velocity_orig = data_orig.get_field(0, 'velocity').data

# Load dealiased velocity data
path_dealiased = '/scratch/mch/fackerma/orders/r2d2/srn/data/tmp/mof/DVD/'
file_dealiased = glob.glob(path_dealiased + 'MLD1916614307L.803')
data_dealiased = pyart.aux_io.read_metranet(file_dealiased[0], reader='python')
velocity_dealiased = data_dealiased.get_field(0, 'velocity').data

# Define radar resolution and grid
resolution = 0.5  # resolution in km per pixel
range_max = velocity_orig.shape[1] * resolution  # maximum range in km
azimuths = np.linspace(0, 360, velocity_orig.shape[0])  # azimuth angles in degrees
ranges = np.arange(0, range_max, resolution)  # radar range bins in km

# Convert polar coordinates (range, azimuth) to geographic coordinates (lat/lon)
def radar_to_latlon(radar_lat, radar_lon, ranges, azimuths):
    R = 6371.0  # Earth radius in kilometers
    azimuths_rad = np.radians(azimuths)
    latitudes = np.zeros((len(azimuths), len(ranges)))
    longitudes = np.zeros((len(azimuths), len(ranges)))

    for i, az in enumerate(azimuths_rad):
        latitudes[i, :] = radar_lat + (ranges / R) * np.cos(az) * (180.0 / np.pi)
        longitudes[i, :] = radar_lon + (ranges / R) * np.sin(az) * (180.0 / np.pi) / np.cos(np.radians(radar_lat))

    return latitudes, longitudes

# Transform radar polar coordinates to lat/lon
lats, lons = radar_to_latlon(radar_lat, radar_lon, ranges, azimuths)

# Create a figure with two subplots
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(18, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Set the background to black for both subplots
for ax in axs:
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

# Create a blue-white-red colormap for velocity data
cmap_velocity = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white", "red"], N=256)
cmap_velocity.set_bad(color='black')

# Plot Dealiased Velocity Data (Left Subplot)
p1 = axs[0].pcolormesh(lons, lats, velocity_dealiased, vmin=-20, vmax=20, cmap=cmap_velocity, transform=ccrs.PlateCarree())
axs[0].set_extent([5, 7.7, 45.5, 47.3], crs=ccrs.PlateCarree())
axs[0].add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
axs[0].add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
axs[0].add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
axs[0].plot(geneva_lon, geneva_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())
axs[0].plot(lausanne_lon, lausanne_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())
axs[0].plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree())
axs[0].set_title('Dealiased Velocity [m s⁻¹]', color='white', loc='right')  # Right-aligned subtitle for dealiased velocity

# Add colorbar for Dealiased Velocity
cbar1 = plt.colorbar(p1, ax=axs[0], cmap=cmap_velocity, extend='both')
cbar1.outline.set_edgecolor('white')
cbar1.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color='white')

# Plot Original Velocity Data (Right Subplot)
p2 = axs[1].pcolormesh(lons, lats, velocity_orig, vmin=-20, vmax=20, cmap=cmap_velocity, transform=ccrs.PlateCarree())
axs[1].set_extent([5, 7.7, 45.5, 47.3], crs=ccrs.PlateCarree())
axs[1].add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
axs[1].add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
axs[1].add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
axs[1].plot(geneva_lon, geneva_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())
axs[1].plot(lausanne_lon, lausanne_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())
axs[1].plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree())
axs[1].set_title('Original Velocity [m s⁻¹]', color='white', loc='right')  # Right-aligned subtitle for original velocity

# Add colorbar for Original Velocity
cbar2 = plt.colorbar(p2, ax=axs[1], cmap=cmap_velocity, extend='both')
cbar2.outline.set_edgecolor('white')
cbar2.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar2.ax.axes, 'yticklabels'), color='white')

# Set main title
plt.suptitle('La Dôle 3rd elevation scan 1430 UTC', color='white', fontsize=16, x=0.5)

# Show the plot
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to avoid overlap and leave space for the title

# Save the figure
#plt.savefig("/users/fackerma/newproject1/figures/velocity_comparison_plot.png")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 0: invalid start byte

In [19]:
# Load dealiased velocity data
path_dealiased = '/scratch/mch/fackerma/orders/r2d2/srn/data/tmp/mof/DVD/'
file_dealiased = glob.glob(path_dealiased + 'DVD1916614307L.803')


#data_dealiased = pyart.aux_io.read_file_py(file_dealiased[0], physic_value = False)
#velocity_dealiased = data_dealiased.get_field(0, 'velocity').data

del_data = pyart.aux_io.read_file_py(file_dealiased[0],physic_value = False)
del_velocity = del_data.datadel_header = del_data.header
nyq=float(del_header['nyquist'])
del_velocity2 = transform_from_digital(del_velocity, nyq)
print(np.unique(del_velocity2))

NameError: name 'del_header' is not defined